In [ ]:
#import required packages
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
import numpy as np

In [ ]:
#load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add layers at the end
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
#set up optimizer, loss and metrics 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#normalize and set up the validation split
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [ ]:
#load train and validation sets
train_generator = train_datagen.flow_from_directory(
    'path to dataset',
    target_size=(224, 224),
    batch_size=32, 
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    'path to dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
#train the model
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
    )

In [ ]:
#save the model
model.save('waste_classification_model.h5')